In [1]:
import pandas as pd
import psycopg

In [2]:
%load_ext sql

In [3]:
%sql postgresql://michellelin@localhost:5432/yelp

In [4]:
def get_db_connection():
    """Establishes and returns a connection to the PostgreSQL database."""
    try:
        conn = psycopg.connect("postgresql://michellelin@localhost/yelp")
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# Denormalized Data

## Create the data schema

In [5]:
# Connect to the PostgreSQL database
conn = get_db_connection()
cursor = conn.cursor()

# Full SQL schema definition
schema_sql = """
DROP TABLE IF EXISTS review CASCADE;
DROP TABLE IF EXISTS checkin CASCADE;
DROP TABLE IF EXISTS tip CASCADE;
DROP TABLE IF EXISTS photo CASCADE;
DROP TABLE IF EXISTS business CASCADE;
DROP TABLE IF EXISTS yelp_user CASCADE;

CREATE TABLE IF NOT EXISTS business (
    business_id VARCHAR(22) PRIMARY KEY,
    name VARCHAR NOT NULL,
    address VARCHAR,
    city VARCHAR,
    state VARCHAR(2),
    postal_code VARCHAR(10),
    latitude NUMERIC,
    longitude NUMERIC,
    stars NUMERIC CHECK (stars BETWEEN 0 AND 5),
    review_count INTEGER,
    is_open INTEGER CHECK (is_open IN (0, 1)),
    attributes TEXT,
    categories TEXT, 
    hours TEXT
);

CREATE TABLE IF NOT EXISTS yelp_user (
    user_id VARCHAR(22) PRIMARY KEY,
    name TEXT,
    review_count INTEGER,
    yelping_since DATE,
    friends TEXT, 
    useful INTEGER,
    funny INTEGER,
    cool INTEGER,
    fans INTEGER,
    elite TEXT, 
    average_stars NUMERIC CHECK (average_stars BETWEEN 0 AND 5),
    compliment_hot INTEGER,
    compliment_more INTEGER,
    compliment_profile INTEGER,
    compliment_cute INTEGER,
    compliment_list INTEGER,
    compliment_note INTEGER,
    compliment_plain INTEGER,
    compliment_cool INTEGER,
    compliment_funny INTEGER,
    compliment_writer INTEGER,
    compliment_photos INTEGER
);

CREATE TABLE IF NOT EXISTS review (
    review_id VARCHAR(22) PRIMARY KEY,
    user_id VARCHAR(22) REFERENCES yelp_user (user_id) ON DELETE CASCADE,
    business_id VARCHAR(22) REFERENCES business (business_id) ON DELETE CASCADE,
    stars NUMERIC CHECK (stars BETWEEN 0 AND 5),
    date DATE NOT NULL,
    text TEXT,
    useful INTEGER,
    funny INTEGER,
    cool INTEGER
);

CREATE TABLE IF NOT EXISTS tip (
    user_id VARCHAR(22) REFERENCES yelp_user (user_id) ON DELETE CASCADE,
    business_id VARCHAR(22) REFERENCES business (business_id) ON DELETE CASCADE,
    text TEXT NOT NULL,
    date DATE NOT NULL,
    compliment_count INTEGER DEFAULT 0,
    PRIMARY KEY (user_id, business_id, date)
);
"""

# Execute the schema to create the tables
cursor.execute(schema_sql)

# Commit and close
conn.commit()
cursor.close()
conn.close()
print("Tables created successfully.")


Tables created successfully.


## Insert data

In [6]:
def bulk_insert(conn, table_name, dataframe, conflict_column=None):
    """
    Insert data from a DataFrame into a PostgreSQL table using psycopg2.
    
    Args:
        conn: Connection object to the database.
        table_name: Name of the target table.
        dataframe: Pandas DataFrame containing the data to be inserted.
        conflict_column: Column(s) to handle ON CONFLICT clause (string or list of columns).
    """
    cursor = conn.cursor()
    try:
        # Create a list of column names for the INSERT statement
        columns = ', '.join([f'"{col}"' for col in dataframe.columns])  # Quote column names to avoid reserved words
        placeholders = ', '.join(['%s'] * len(dataframe.columns))
        
        # Create the SQL insert query
        insert_query = f"""
        INSERT INTO {table_name} ({columns}) 
        VALUES ({placeholders})
        """
        
        # Add ON CONFLICT clause
        if conflict_column:
            if isinstance(conflict_column, list):  # If it's a list of columns, create composite key
                conflict_column = ', '.join([f'"{col}"' for col in conflict_column])  # Add quotes
            else:
                conflict_column = f'"{conflict_column}"'  # Handle single column key
            
            insert_query += f"""
            ON CONFLICT ({conflict_column}) DO NOTHING
            """
        
        # Use executemany to insert all the rows
        cursor.executemany(insert_query, dataframe.values.tolist())
        
        # Commit the changes
        conn.commit()
        
        print(f"Successfully inserted {len(dataframe)} rows into {table_name}.")
    except Exception as e:
        conn.rollback()
        print(f"Error inserting data into {table_name}: {e}")
    finally:
        cursor.close()


In [7]:

# def bulk_insert(conn, table_name, dataframe, conflict_column=None):
#     """
#     Inserts data from a Pandas DataFrame into a PostgreSQL table.

#     Args:
#         conn: PostgreSQL connection object.
#         table_name: The name of the PostgreSQL table to insert into.
#         dataframe: The Pandas DataFrame containing the data to be inserted.
#         conflict_column: The column name to use in the ON CONFLICT clause.
#     """
#     try:
#         # Get column names from the DataFrame
#         columns = ', '.join(dataframe.columns)
        
#         # Create a placeholder string for the VALUES portion of the SQL
#         placeholders = ', '.join(['%s'] * len(dataframe.columns))
        
#         # Full SQL insert query with dynamic conflict column
#         if conflict_column:
#             query = f"""
#             INSERT INTO {table_name} ({columns}) 
#             VALUES ({placeholders}) 
#             ON CONFLICT ({conflict_column}) DO NOTHING;
#             """
#         else:
#             query = f"""
#             INSERT INTO {table_name} ({columns}) 
#             VALUES ({placeholders});
#             """
        
#         # Convert DataFrame to a list of tuples
#         data = [tuple(x) for x in dataframe.to_numpy()]
        
#         with conn.cursor() as cursor:
#             cursor.executemany(query, data)
        
#         # Commit the changes
#         conn.commit()
#         print(f"Successfully inserted {len(data)} rows into {table_name}.")
        
#     except Exception as e:
#         print(f"Error inserting data into {table_name}: {e}")

### business

In [8]:
# Step 1: Read the CSV file for 'business'
df_business = pd.read_csv('../sampled_data/business_sample.csv')

# Convert columns to plain text
df_business['attributes'] = df_business['attributes'].astype(str)
df_business['categories'] = df_business['categories'].astype(str)
df_business['hours'] = df_business['hours'].astype(str)

# Replace NaN or None with NULL in the DataFrame for columns that allow NULL
df_business['attributes'] = df_business['attributes'].replace({pd.NA: None, 'nan': None})
df_business['categories'] = df_business['categories'].replace({pd.NA: None, 'nan': None})
df_business['hours'] = df_business['hours'].replace({pd.NA: None, 'nan': None})

print(df_business.head())


              business_id                              name  \
0  yFrOuce72KhvW0LpU5FY6A            Earnest Bar & Hideaway   
1  WfGXVT-WOgDymYv_kL28GQ  Vanessa Cafe Restaraunt Pizzeria   
2  otm8R7rkCtCNM0g0TC55-Q                   The Gables Cafe   
3  3xMtsSQ42lBpkP-R3726lw                     Pinky's Nails   
4  ne1EabaPSD9WjIpWZ26JMQ        Reno Tahoe Window Cleaning   

                            address                   city state postal_code  \
0           438 Houston St, Ste 160              Nashville    TN       37203   
1                    3815 Church Rd  Mount Laurel Township    NJ       08054   
2                 4600 Woodland Ave           Philadelphia    PA       19143   
3                      63 Boone Vlg             Zionsville    IN       46077   
4  59 Damonte Ranch Pkwy, Ste B-187                   Reno    NV       89521   

    latitude   longitude  stars  review_count  is_open  \
0  36.143036  -86.767485    4.0           265        1   
1  39.928964  -74.970330

In [9]:
# Connect to the PostgreSQL database
conn = get_db_connection()

# Call the bulk_insert function to load the data
bulk_insert(conn, 'business', df_business)

# Close the connection
conn.close()

Successfully inserted 1000 rows into business.


In [10]:
%%sql
SELECT * FROM business LIMIT 1;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

1 rows affected.

business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
yFrOuce72KhvW0LpU5FY6A,Earnest Bar & Hideaway,"438 Houston St, Ste 160",Nashville,TN,37203,36.143036,-86.767485,4,265,1,"{'WheelchairAccessible': 'True', 'HasTV': 'True', 'BikeParking': 'True', 'Music': ""{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}"", 'RestaurantsReservations': 'True', 'NoiseLevel': ""u'average'"", 'Corkage': 'True', 'CoatCheck': 'False', 'Caters': 'True', 'RestaurantsAttire': ""'casual'"", 'BusinessParking': ""{'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False}"", 'DogsAllowed': 'False', 'GoodForDancing': 'False', 'Ambience': ""{'touristy': False, 'hipster': True, 'romantic': True, 'divey': False, 'intimate': True, 'trendy': True, 'upscale': False, 'classy': True, 'casual': False}"", 'RestaurantsTakeOut': 'True', 'HappyHour': 'True', 'RestaurantsTableService': 'True', 'RestaurantsPriceRange2': '2', 'Alcohol': ""u'full_bar'"", 'OutdoorSeating': 'True', 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'False', 'BusinessAcceptsBitcoin': 'False', 'BestNights': ""{'monday': False, 'tuesday': True, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}"", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'WiFi': ""u'free'"", 'Smoking': ""u'no'"", 'GoodForMeal': ""{u'breakfast': False, u'brunch': True, u'lunch': True, u'dinner': True, u'latenight': None, u'dessert': None}""}","American (New), American (Traditional), Nightlife, Restaurants, Bars, Sandwiches, Cocktail Bars","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', 'Wednesday': '17:0-22:0', 'Thursday': '17:0-22:0', 'Friday': '17:0-23:0', 'Saturday': '17:0-22:0', 'Sunday': '10:0-15:0'}"


### yelp_user

In [11]:
# Step 1: Read the CSV file for 'yelp_user'
df_user = pd.read_csv('../sampled_data/users_sample.csv')

# Step 2: Connect to the PostgreSQL database
conn = get_db_connection()

# Step 3: Insert data into the 'yelp_user' table
bulk_insert(conn, 'yelp_user', df_user, conflict_column='user_id')


Successfully inserted 38921 rows into yelp_user.


In [12]:
%%sql
SELECT COUNT(*) FROM yelp_user;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

1 rows affected.

count
38921


### review

In [13]:
# Step 1: Read the CSV file for 'reviews_sample'
df_review = pd.read_csv('../sampled_data/reviews_sample.csv')

# Step 2: Connect to the PostgreSQL database
conn = get_db_connection()

# Step 3: Insert data into the 'yelp_user' table
bulk_insert(conn, 'review', df_review, conflict_column='review_id')

Successfully inserted 44650 rows into review.


In [14]:
%%sql
SELECT COUNT(*) FROM review;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

1 rows affected.

count
44650


In [15]:
%%sql
SELECT * FROM review LIMIT 1;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

1 rows affected.

review_id,user_id,business_id,stars,date,text,useful,funny,cool
hOwlMPilziEHtyuKpf4Rlg,ubOMW9s0Bhi_GGPeG8Uq7g,hy5GpGXAna-5qrb3zNub6g,5,2017-03-27,Eat here at least 2 times a week. Servers are always friendly and the food is amazing. I never leave disappointed & always leave with a full stomach,0,1,0


In [16]:
# def bulk_insert(conn, table_name, dataframe, conflict_column=None):
#     """
#     Insert data from a DataFrame into a PostgreSQL table using psycopg2.
    
#     Args:
#         conn: Connection object to the database.
#         table_name: Name of the target table.
#         dataframe: Pandas DataFrame containing the data to be inserted.
#         conflict_column: Column(s) to handle ON CONFLICT clause (string or list of columns).
#     """
#     cursor = conn.cursor()
#     try:
#         # Create a list of column names for the INSERT statement
#         columns = ', '.join([f'"{col}"' for col in dataframe.columns])  # Quote column names to avoid reserved words
#         placeholders = ', '.join(['%s'] * len(dataframe.columns))
        
#         # Create the SQL insert query
#         insert_query = f"""
#         INSERT INTO {table_name} ({columns}) 
#         VALUES ({placeholders})
#         """
        
#         # Add ON CONFLICT clause
#         if conflict_column:
#             if isinstance(conflict_column, list):  # If it's a list of columns, create composite key
#                 conflict_column = ', '.join([f'"{col}"' for col in conflict_column])  # Add quotes
#             else:
#                 conflict_column = f'"{conflict_column}"'  # Handle single column key
            
#             insert_query += f"""
#             ON CONFLICT ({conflict_column}) DO NOTHING
#             """
        
#         # Use executemany to insert all the rows
#         cursor.executemany(insert_query, dataframe.values.tolist())
        
#         # Commit the changes
#         conn.commit()
        
#         print(f"Successfully inserted {len(dataframe)} rows into {table_name}.")
#     except Exception as e:
#         conn.rollback()
#         print(f"Error inserting data into {table_name}: {e}")
#     finally:
#         cursor.close()


In [17]:
# df_checkin = pd.read_csv('../sampled_data/checkins_sample.csv')
# df_checkin.rename(columns={'date': 'date_time'}, inplace=True)
# conn = get_db_connection()
# bulk_insert(conn, 'checkin', df_checkin, conflict_column=['business_id', 'date_time'])

### tip

In [18]:
# Step 1: Read the CSV file
df_tip = pd.read_csv('../sampled_data/tips_sample.csv')

# Step 2: Connect to the PostgreSQL database
conn = get_db_connection()

# Step 3: Insert data into the table
bulk_insert(conn, 'tip', df_tip, conflict_column=['user_id', 'business_id', 'date'])

Successfully inserted 3042 rows into tip.


In [19]:
%%sql
SELECT * FROM tip LIMIT 1;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

1 rows affected.

user_id,business_id,text,date,compliment_count
LkXNQQaIc5yrKu9Ap87RrA,SekWxm92phAL_P_KySUnxA,Fried Pickles are good.,2011-10-12,0


# Normalized Data

## Explore business data

In [20]:
# Connect to the PostgreSQL database
conn = get_db_connection()
cursor = conn.cursor()

# Full SQL schema definition
schema_sql = """
DROP TABLE IF EXISTS location CASCADE;
DROP TABLE IF EXISTS nor_business CASCADE;

CREATE TABLE IF NOT EXISTS nor_business (
    business_id VARCHAR(22) PRIMARY KEY,
    name VARCHAR NOT NULL,
    stars NUMERIC CHECK (stars BETWEEN 0 AND 5),
    review_count INTEGER,
    is_open INTEGER CHECK (is_open IN (0, 1)),
    attributes TEXT,
    categories TEXT, 
    hours TEXT
);

CREATE TABLE IF NOT EXISTS location (
    business_id VARCHAR(22) REFERENCES nor_business (business_id) ON DELETE CASCADE,
    address VARCHAR,
    city VARCHAR,
    state VARCHAR(2),
    postal_code VARCHAR(10),
    latitude NUMERIC,
    longitude NUMERIC
);

"""

# Execute the schema to create the tables
cursor.execute(schema_sql)

# Commit and close
conn.commit()
cursor.close()
conn.close()
print("Tables created successfully.")


Tables created successfully.


In [21]:
## normalized business
# Step 1: Read the CSV file
df_nor_business = df_business[['business_id', 'name', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']]
df_nor_business

# Step 2: Connect to the PostgreSQL database
conn = get_db_connection()

# Step 3: Insert data into the table
bulk_insert(conn, 'nor_business', df_nor_business, conflict_column=['business_id'])

Successfully inserted 1000 rows into nor_business.


In [22]:
%%sql
SELECT * FROM nor_business LIMIT 5;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

5 rows affected.

business_id,name,stars,review_count,is_open,attributes,categories,hours
yFrOuce72KhvW0LpU5FY6A,Earnest Bar & Hideaway,4,265,1,"{'WheelchairAccessible': 'True', 'HasTV': 'True', 'BikeParking': 'True', 'Music': ""{'dj': False, 'background_music': False, 'no_music': False, 'jukebox': False, 'live': False, 'video': False, 'karaoke': False}"", 'RestaurantsReservations': 'True', 'NoiseLevel': ""u'average'"", 'Corkage': 'True', 'CoatCheck': 'False', 'Caters': 'True', 'RestaurantsAttire': ""'casual'"", 'BusinessParking': ""{'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False}"", 'DogsAllowed': 'False', 'GoodForDancing': 'False', 'Ambience': ""{'touristy': False, 'hipster': True, 'romantic': True, 'divey': False, 'intimate': True, 'trendy': True, 'upscale': False, 'classy': True, 'casual': False}"", 'RestaurantsTakeOut': 'True', 'HappyHour': 'True', 'RestaurantsTableService': 'True', 'RestaurantsPriceRange2': '2', 'Alcohol': ""u'full_bar'"", 'OutdoorSeating': 'True', 'BusinessAcceptsCreditCards': 'True', 'GoodForKids': 'False', 'BusinessAcceptsBitcoin': 'False', 'BestNights': ""{'monday': False, 'tuesday': True, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}"", 'RestaurantsDelivery': 'True', 'RestaurantsGoodForGroups': 'True', 'WiFi': ""u'free'"", 'Smoking': ""u'no'"", 'GoodForMeal': ""{u'breakfast': False, u'brunch': True, u'lunch': True, u'dinner': True, u'latenight': None, u'dessert': None}""}","American (New), American (Traditional), Nightlife, Restaurants, Bars, Sandwiches, Cocktail Bars","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', 'Wednesday': '17:0-22:0', 'Thursday': '17:0-22:0', 'Friday': '17:0-23:0', 'Saturday': '17:0-22:0', 'Sunday': '10:0-15:0'}"
WfGXVT-WOgDymYv_kL28GQ,Vanessa Cafe Restaraunt Pizzeria,5,15,1,"{'RestaurantsReservations': 'True', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsTakeOut': 'True', 'RestaurantsTableService': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"", 'RestaurantsDelivery': 'True', 'Caters': 'True'}","Pizza, Italian, Restaurants, Sicilian","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22:0', 'Thursday': '11:0-22:0', 'Friday': '11:0-22:0', 'Saturday': '11:0-22:0', 'Sunday': '11:0-22:0'}"
otm8R7rkCtCNM0g0TC55-Q,The Gables Cafe,5,14,1,"{'RestaurantsTableService': 'True', 'WiFi': ""u'free'"", 'RestaurantsDelivery': 'None', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsTakeOut': 'True', 'BYOB': 'False', 'OutdoorSeating': 'True', 'Caters': 'True', 'WheelchairAccessible': 'True', 'Alcohol': ""u'none'"", 'BikeParking': 'True', 'Corkage': 'False', 'HasTV': 'False'}","Food, Coffee & Tea, Restaurants, Italian, Desserts, Breakfast & Brunch, Sandwiches, Bakeries",None
3xMtsSQ42lBpkP-R3726lw,Pinky's Nails,3.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '2', 'BikeParking': 'True', 'BusinessParking': ""{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}""}","Nail Salons, Beauty & Spas","{'Monday': '9:30-19:30', 'Tuesday': '9:30-19:30', 'Wednesday': '9:30-19:30', 'Thursday': '9:30-19:30', 'Friday': '9:30-19:30', 'Saturday': '9:0-19:0', 'Sunday': '11:30-17:0'}"
ne1EabaPSD9WjIpWZ26JMQ,Reno Tahoe Window Cleaning,4.5,94,1,"{'BusinessAcceptsCreditCards': 'True', 'BikeParking': 'True', 'WheelchairAccessible': 'True', 'ByAppointmentOnly': 'True'}","Pest Control, Local Services, Home Services, Window Washing, Pressure Washers, Damage Restoration, Home Cleaning, Gutter Services, Shopping, Electricians, Holiday Decorations, Holiday Decorating Services, Home & Garden","{'Monday': '0:0-0:0', 'Tuesday': '8:0-17:0', 'Wednesday': '8:0-17:0', 'Thursday': '8:0-17:0', 'Friday': '8:0-17:0', 'Saturday': '8:0-17:0', 'Sunday': '8:0-17:0'}"


In [23]:
%%sql
EXPLAIN ANALYZE SELECT * FROM nor_business LIMIT 5;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

4 rows affected.

QUERY PLAN
Limit (cost=0.00..0.21 rows=5 width=230) (actual time=0.014..0.015 rows=5 loops=1)
-> Seq Scan on nor_business (cost=0.00..104.80 rows=2480 width=230) (actual time=0.013..0.013 rows=5 loops=1)
Planning Time: 0.065 ms
Execution Time: 0.029 ms


In [24]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [25]:
## location
df_location = df_business[['business_id', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude']]
df_location

# Step 2: Connect to the PostgreSQL database
conn = get_db_connection()

# Step 3: Insert data into the table
bulk_insert(conn, 'location', df_location)

Successfully inserted 1000 rows into location.


In [26]:
%%sql
SELECT * FROM location LIMIT 10;

Running query in 'postgresql://michellelin@localhost:5432/yelp'

10 rows affected.

business_id,address,city,state,postal_code,latitude,longitude
yFrOuce72KhvW0LpU5FY6A,"438 Houston St, Ste 160",Nashville,TN,37203,36.143036,-86.767485
WfGXVT-WOgDymYv_kL28GQ,3815 Church Rd,Mount Laurel Township,NJ,08054,39.9289637,-74.9703303
otm8R7rkCtCNM0g0TC55-Q,4600 Woodland Ave,Philadelphia,PA,19143,39.9441938,-75.2097087
3xMtsSQ42lBpkP-R3726lw,63 Boone Vlg,Zionsville,IN,46077,39.951867,-86.278108
ne1EabaPSD9WjIpWZ26JMQ,"59 Damonte Ranch Pkwy, Ste B-187",Reno,NV,89521,39.4202983,-119.7569454
u8wxP8s-jr0KCZTfGMl1UQ,212 Brown St,Philadelphia,PA,19123,39.9630155,-75.1417047
WOY6dk3JeyP-bYoELP_MIQ,5110 Meadowood Mall Cir,Reno,NV,89502,39.4739038,-119.7828905
zXgksmhdzgL4Xx5FylbP3w,300 W Mt Pleasant Ave,Philadelphia,PA,19119,40.0539069,-75.1942308
8-j10_I745ITrczbkfD-Pw,145 S Gulph Rd,King Of Prussia,PA,19406,40.0841953351,-75.3873682022
rDg7qsQwE55xxJiU65_N7A,1808 W End Ave,Nashville,TN,37203,36.15287,-86.797082


## Explore yelp users data

In [27]:
df_user.isnull().sum()
# from the result we can see there are a lot of null values for elite, so we can normalize that as well

user_id                   0
name                      0
review_count              0
yelping_since             0
useful                    0
funny                     0
cool                      0
elite                 31343
friends                   0
fans                      0
average_stars             0
compliment_hot            0
compliment_more           0
compliment_profile        0
compliment_cute           0
compliment_list           0
compliment_note           0
compliment_plain          0
compliment_cool           0
compliment_funny          0
compliment_writer         0
compliment_photos         0
dtype: int64

In [28]:
# Step 1: Split the 'elite' column into multiple rows
df_elite = df_user.assign(elite=df_user['elite'].str.split(',')).explode('elite')

# Step 2: Remove any extra spaces and drop NaN years
df_elite['elite'] = df_elite['elite'].str.strip()
df_elite = df_elite.dropna(subset=['elite'])

# Step 3: Drop duplicates in case of redundancy
df_elite = df_elite[['user_id', 'elite']].drop_duplicates()

# Step 4: Rename 'elite' to 'elite_year' 
df_elite.rename(columns={'elite': 'elite_year'}, inplace=True)

print(df_elite)


                      user_id elite_year
0      j14WgRoU_-2ZE1aw1dXrJg       2009
0      j14WgRoU_-2ZE1aw1dXrJg       2010
0      j14WgRoU_-2ZE1aw1dXrJg       2011
0      j14WgRoU_-2ZE1aw1dXrJg       2012
0      j14WgRoU_-2ZE1aw1dXrJg       2013
...                       ...        ...
38072  6kHOI15vOLMgL4wkBo8Jsg       2018
38118  5EbR_Htqm1iy34mRUThvPg       2015
38138  pZBydccLh3R0LVAycZiE3Q       2019
38138  pZBydccLh3R0LVAycZiE3Q         20
38138  pZBydccLh3R0LVAycZiE3Q       2021

[32493 rows x 2 columns]


In [29]:
### friends
# Normalize 'friends' column
df_friends = df_user[['user_id', 'friends']].copy()

# Split the 'friends' column by comma and explode into multiple rows
df_friends = df_friends.assign(friends=df_friends['friends'].str.split(',')).explode('friends')

# Clean: Remove extra spaces, NaN, and empty strings
df_friends['friends'] = df_friends['friends'].str.strip()  # Strip spaces
df_friends = df_friends.dropna(subset=['friends'])  # Drop NaN
df_friends = df_friends[df_friends['friends'] != '']  # Drop empty strings
df_friends = df_friends[df_friends['friends'] != 'None']

df_friends.rename(columns={'friends': 'friend_id'}, inplace=True)

print(df_friends)

                      user_id               friend_id
0      j14WgRoU_-2ZE1aw1dXrJg  ueRPE0CX75ePGMqOFVj6IQ
0      j14WgRoU_-2ZE1aw1dXrJg  52oH4DrRvzzl8wh5UXyU0A
0      j14WgRoU_-2ZE1aw1dXrJg  E_GAXhVA1_lVC2aFpMQElA
0      j14WgRoU_-2ZE1aw1dXrJg  HwlpkOpidkZWvyjrxFk6Ag
0      j14WgRoU_-2ZE1aw1dXrJg  kuDmRGcvJhFCHEXTNH1d4Q
...                       ...                     ...
38353  SvVxrFoegERNTxtRdbkHLg  uiNDEOR0I-ElPzEAqxKa2A
38362  ara-bY68Rqk8er4zM2v3GA  05OIkALsFYK34rVCB-uwzw
38441  4Ilu9urihE_Q0sDfQIkZAQ  lOroVqramCWeA91J11w1DA
38466  a9RPyOIgkv_MSLT9KHIeyQ  xMhG6bdjxVAuMqQ7g3Fm7w
38616  QpUNYE8fD07YFvcP-HLn7g  VwZ8aCKA51kELLmseFL54g

[3545950 rows x 2 columns]


In [30]:
# Connect to the PostgreSQL database
conn = get_db_connection()
cursor = conn.cursor()

# Full SQL schema definition
schema_sql = """
DROP TABLE IF EXISTS friends CASCADE;
DROP TABLE IF EXISTS elite CASCADE;
DROP TABLE IF EXISTS elite_year CASCADE;
DROP TABLE IF EXISTS nor_user CASCADE;

CREATE TABLE IF NOT EXISTS nor_user (
    user_id VARCHAR(22) PRIMARY KEY,
    name TEXT,
    review_count INTEGER,
    yelping_since DATE,
    useful INTEGER,
    funny INTEGER,
    cool INTEGER,
    fans INTEGER,
    average_stars NUMERIC CHECK (average_stars BETWEEN 0 AND 5),
    compliment_hot INTEGER,
    compliment_more INTEGER,
    compliment_profile INTEGER,
    compliment_cute INTEGER,
    compliment_list INTEGER,
    compliment_note INTEGER,
    compliment_plain INTEGER,
    compliment_cool INTEGER,
    compliment_funny INTEGER,
    compliment_writer INTEGER,
    compliment_photos INTEGER
);

CREATE TABLE IF NOT EXISTS friends (
    user_id VARCHAR(22) REFERENCES nor_user(user_id) ON DELETE CASCADE,
    friend_id VARCHAR(22)
);

CREATE TABLE IF NOT EXISTS elite (
    user_id VARCHAR(22) REFERENCES nor_user(user_id) ON DELETE CASCADE,
    elite_year INTEGER
);


"""

# Execute the schema to create the tables
cursor.execute(schema_sql)

# Commit and close
conn.commit()
cursor.close()
conn.close()
print("Tables created successfully.")

Tables created successfully.


### Insert into normalized users data

In [31]:
## insert nor_user
df_user_nor = df_user[['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos']]

# Step 2: Connect to the PostgreSQL database
conn = get_db_connection()

# Step 3: Insert data into the table
bulk_insert(conn, 'nor_user', df_user_nor, conflict_column='user_id')

Successfully inserted 38921 rows into nor_user.


In [32]:
df_elite.columns

Index(['user_id', 'elite_year'], dtype='object')

In [33]:
## insert elite
conn = get_db_connection()
bulk_insert(conn, 'elite', df_elite)

Successfully inserted 32493 rows into elite.


In [34]:
## insert friends
conn = get_db_connection()
bulk_insert(conn, 'friends', df_friends)

Successfully inserted 3545950 rows into friends.


In [35]:
cursor.close()
conn.close()